In [1]:
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("kiwisolver==1.2.0")
sc.install_pypi_package("matplotlib==3.1.1")
sc.install_pypi_package("chart_studio")
sc.install_pypi_package("iso3166")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1636441302464_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/31/b9/6202dcae729998a0ade30e80ac00f616542ef445b088ec970d407dfd41c0/kiwisolver-1.2.0-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/19/7a/60bd79c5d79559150f8bba866dd7d434f0a170312e4d15e8aefa5faba294/matplotlib-3.1.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f1/10/313669b34fc25319a171b2a3cdec78da4c6b6a29e7bea200c03755bb6396/pyparsing-3.0.5-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/f9/695d6bedebd747e5eb0fe8fad57b72fdf25411273a39791cde838d5a8f51/cycler-0.11.0-py3-none-any.whl

  Using cac

In [7]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import numpy as np
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark import *
import regex
import time

#   https://registry.opendata.aws/openaq/
conf = SparkConf()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
smog_df = spark.read.json("s3a://openaq-fetches/realtime/2019-01*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
for i in range(3):
    start = time.time() 
    data = smog_df.where(col('parameter')=='pm10')

    berlin = (data.where((col('city')=='Berlin'))
          .agg(max(col("value")))
          .take(1)[0][0]
    )

    print('Berlin:',berlin)

    df = (data.where(col('Country')=='PL')
          .groupBy('city').agg(avg("value").alias('mean'))
         .where(col('mean') < berlin)
          .select('city','mean')
          .orderBy('city')
          .show()
    )

    end = time.time() - start
    print(end) # seconds

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Berlin: 59.94
+-------------------+------------------+
|               city|              mean|
+-------------------+------------------+
|           Augustów|24.675560675883258|
|     Biała Podlaska|40.489652605459064|
|          Białystok|25.732748201438856|
|      Bielsko-Biała| 54.62891449213161|
|       Borsukowizna| 20.85145228215768|
|           Borówiec|26.703827871759895|
|          Bydgoszcz|33.552165458937196|
|              Chełm|  53.1921760391198|
|            Cieszyn| 49.85524396848138|
|        Częstochowa|46.235271107064854|
|     Duszniki-Zdrój|41.700841172839496|
|         Działoszyn|23.252095103163686|
|        Dzierżoniów|  39.8659613061798|
|   Dąbrowa Górnicza|56.166497768240355|
|             Elbląg|28.299747942386823|
|                Ełk| 29.96607197260275|
|              Gajew|33.510316368638236|
|             Gdańsk|  21.0576575455483|
|             Gdynia|19.889729154667908|
|Gorzów Wielkopolski|23.122344225759765|
+-------------------+------------------+
on